# Spaceship Titanic: Logistic Regression
## Imports

In [1]:
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
warnings.simplefilter(action="ignore", category=FutureWarning)

## Read data

In [3]:
data_dir = Path.cwd().parent / "input" / "spaceship-titanic"
assert data_dir.exists(), f"directory doesn't exist: {data_dir}"

In [4]:
df = pd.read_csv(
    data_dir / "train_imputed.csv",
    index_col="PassengerId",
    dtype={"CompCntReduced": pd.CategoricalDtype(categories=["0", "1", "2", "3+"], ordered=True)},
).assign(CompCntReduced=lambda x: x.CompCntReduced.cat.codes)
df.head(10)

,Alone,CompCntReduced,HomePlanetOrd,CryoSleep,CabinDeckOrd,CabinPort,DestinationOrd,DiscretizedAge4,DiscretizedAge5,VIP,PosRoomService,PosFoodCourt,PosShoppingMall,PosSpa,PosVRDeck,PTTotalSpent,Transported
PassengerId,,,,,,,,,,,,,,,,,
0001_01,True,0,1,False,0,True,2,3,3,False,False,False,False,False,False,-1.140331,False
0002_01,True,0,0,False,3,False,2,1,2,False,True,True,True,True,True,0.576497,True
0003_01,False,1,1,False,5,False,2,3,4,True,True,True,False,True,True,1.501611,False
0003_02,False,1,1,False,5,False,2,2,3,False,False,True,True,True,True,1.242461,False
0004_01,True,0,0,False,3,False,2,0,0,False,True,True,True,True,True,0.704061,True
0005_01,True,0,0,False,3,True,1,3,4,False,False,True,False,True,False,0.592626,True
0006_01,False,1,0,False,3,False,2,1,2,False,True,True,True,False,False,0.828008,True
0006_02,False,1,0,True,4,False,2,2,2,False,False,False,False,False,False,-1.140331,True
0007_01,True,0,0,False,3,False,2,2,3,False,False,True,True,True,False,0.681374,True


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Alone            8693 non-null   bool   
 1   CompCntReduced   8693 non-null   int8   
 2   HomePlanetOrd    8693 non-null   int64  
 3   CryoSleep        8693 non-null   bool   
 4   CabinDeckOrd     8693 non-null   int64  
 5   CabinPort        8693 non-null   bool   
 6   DestinationOrd   8693 non-null   int64  
 7   DiscretizedAge4  8693 non-null   int64  
 8   DiscretizedAge5  8693 non-null   int64  
 9   VIP              8693 non-null   bool   
 10  PosRoomService   8693 non-null   bool   
 11  PosFoodCourt     8693 non-null   bool   
 12  PosShoppingMall  8693 non-null   bool   
 13  PosSpa           8693 non-null   bool   
 14  PosVRDeck        8693 non-null   bool   
 15  PTTotalSpent     8693 non-null   float64
 16  Transported      8693 non-null   bool   
dtypes: bool(10

## Categorical features: One-hot encoding
Figuring out how I'm going to do this:

In [6]:
feat = "HomePlanetOrd"
uniq_vals: list[int] = np.sort(df[feat].unique()).tolist()

In [7]:
encoder = OneHotEncoder(
    categories=[uniq_vals],  # pyright: ignore [reportArgumentType]
    sparse_output=False,
    dtype=np.int8,
)
one_hot_encoded = encoder.fit_transform(df[[feat]])

In [8]:
new_cols = [f"{feat}_{val}" for val in uniq_vals]
df_one_hot = pd.DataFrame(one_hot_encoded, columns=new_cols, index=df.index)

In [9]:
df = df.merge(df_one_hot, left_index=True, right_index=True)
df[[feat, *new_cols]].head(20)

,HomePlanetOrd,HomePlanetOrd_0,HomePlanetOrd_1,HomePlanetOrd_2
PassengerId,,,,
0001_01,1,0,1,0
0002_01,0,1,0,0
0003_01,1,0,1,0
0003_02,1,0,1,0
0004_01,0,1,0,0
0005_01,0,1,0,0
0006_01,0,1,0,0
0006_02,0,1,0,0
0007_01,0,1,0,0


In [10]:
df = df.drop(columns=feat)

Turn the above code into a function:

In [11]:
def one_hot_encode(df: pd.DataFrame, feat: str) -> pd.DataFrame:
    uniq_vals = np.sort(df[feat].unique()).tolist()

    encoder = OneHotEncoder(
        categories=[uniq_vals],  # pyright: ignore [reportArgumentType]
        sparse_output=False,
        dtype=np.int8,
    )
    df_one_hot = pd.DataFrame(
        encoder.fit_transform(df[[feat]]),
        columns=[f"{feat}_{val}" for val in uniq_vals],
        index=df.index,
    )

    df = df.merge(df_one_hot, left_index=True, right_index=True).drop(columns=feat)
    return df

In [12]:
cat_cols = [
    "CompCntReduced",
    "CabinDeckOrd",
    "DestinationOrd",
    "DiscretizedAge4",
    "DiscretizedAge5",
]
for col in cat_cols:
    df = one_hot_encode(df, col)

In [13]:
# Re-order DataFrame columns
cols = [
    "Alone",
    "CompCntReduced_0",
    "CompCntReduced_1",
    "CompCntReduced_2",
    "CompCntReduced_3",
    "HomePlanetOrd_0",
    "HomePlanetOrd_1",
    "HomePlanetOrd_2",
    "CryoSleep",
    "CabinDeckOrd_0",
    "CabinDeckOrd_1",
    "CabinDeckOrd_2",
    "CabinDeckOrd_3",
    "CabinDeckOrd_4",
    "CabinDeckOrd_5",
    "CabinPort",
    "DestinationOrd_0",
    "DestinationOrd_1",
    "DestinationOrd_2",
    "DiscretizedAge4_0",
    "DiscretizedAge4_1",
    "DiscretizedAge4_2",
    "DiscretizedAge4_3",
    "DiscretizedAge5_0",
    "DiscretizedAge5_1",
    "DiscretizedAge5_2",
    "DiscretizedAge5_3",
    "DiscretizedAge5_4",
    "VIP",
    "PosRoomService",
    "PosFoodCourt",
    "PosShoppingMall",
    "PosSpa",
    "PosVRDeck",
    "PTTotalSpent",
    "Transported",
]
df = df[cols]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Alone              8693 non-null   bool   
 1   CompCntReduced_0   8693 non-null   int8   
 2   CompCntReduced_1   8693 non-null   int8   
 3   CompCntReduced_2   8693 non-null   int8   
 4   CompCntReduced_3   8693 non-null   int8   
 5   HomePlanetOrd_0    8693 non-null   int8   
 6   HomePlanetOrd_1    8693 non-null   int8   
 7   HomePlanetOrd_2    8693 non-null   int8   
 8   CryoSleep          8693 non-null   bool   
 9   CabinDeckOrd_0     8693 non-null   int8   
 10  CabinDeckOrd_1     8693 non-null   int8   
 11  CabinDeckOrd_2     8693 non-null   int8   
 12  CabinDeckOrd_3     8693 non-null   int8   
 13  CabinDeckOrd_4     8693 non-null   int8   
 14  CabinDeckOrd_5     8693 non-null   int8   
 15  CabinPort          8693 non-null   bool   
 16  DestinationOrd_0   8

In [14]:
# Convert boolean columns
bool_cols = df.select_dtypes(include=[bool]).columns.values.tolist()  # pyright: ignore [reportArgumentType]
df.loc[:, bool_cols] = df[bool_cols].astype(np.int8)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Alone              8693 non-null   int8   
 1   CompCntReduced_0   8693 non-null   int8   
 2   CompCntReduced_1   8693 non-null   int8   
 3   CompCntReduced_2   8693 non-null   int8   
 4   CompCntReduced_3   8693 non-null   int8   
 5   HomePlanetOrd_0    8693 non-null   int8   
 6   HomePlanetOrd_1    8693 non-null   int8   
 7   HomePlanetOrd_2    8693 non-null   int8   
 8   CryoSleep          8693 non-null   int8   
 9   CabinDeckOrd_0     8693 non-null   int8   
 10  CabinDeckOrd_1     8693 non-null   int8   
 11  CabinDeckOrd_2     8693 non-null   int8   
 12  CabinDeckOrd_3     8693 non-null   int8   
 13  CabinDeckOrd_4     8693 non-null   int8   
 14  CabinDeckOrd_5     8693 non-null   int8   
 15  CabinPort          8693 non-null   int8   
 16  DestinationOrd_0   8

In [15]:
df = df.astype(np.float_)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Alone              8693 non-null   float64
 1   CompCntReduced_0   8693 non-null   float64
 2   CompCntReduced_1   8693 non-null   float64
 3   CompCntReduced_2   8693 non-null   float64
 4   CompCntReduced_3   8693 non-null   float64
 5   HomePlanetOrd_0    8693 non-null   float64
 6   HomePlanetOrd_1    8693 non-null   float64
 7   HomePlanetOrd_2    8693 non-null   float64
 8   CryoSleep          8693 non-null   float64
 9   CabinDeckOrd_0     8693 non-null   float64
 10  CabinDeckOrd_1     8693 non-null   float64
 11  CabinDeckOrd_2     8693 non-null   float64
 12  CabinDeckOrd_3     8693 non-null   float64
 13  CabinDeckOrd_4     8693 non-null   float64
 14  CabinDeckOrd_5     8693 non-null   float64
 15  CabinPort          8693 non-null   float64
 16  DestinationOrd_0   8

In [16]:
df.head(20)

,Alone,CompCntReduced_0,CompCntReduced_1,CompCntReduced_2,CompCntReduced_3,HomePlanetOrd_0,HomePlanetOrd_1,HomePlanetOrd_2,CryoSleep,CabinDeckOrd_0,...,DiscretizedAge5_3,DiscretizedAge5_4,VIP,PosRoomService,PosFoodCourt,PosShoppingMall,PosSpa,PosVRDeck,PTTotalSpent,Transported
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.140331,0.0
0002_01,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.576497,1.0
0003_01,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.501611,0.0
0003_02,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.242461,0.0
0004_01,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.704061,1.0
0005_01,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.592626,1.0
0006_01,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.828008,1.0
0006_02,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.140331,1.0
0007_01,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.681374,1.0


In [17]:
# feat_cols
# X = df[feat_cols]
# y = df["Transported"]